In [25]:
import requests
from google.cloud import bigquery
from datetime import datetime

In [26]:
def send_slack_message(webhook_url, message):

    message['time'] = datetime.now().strftime("%I:%M %p")

    response = requests.post(
        webhook_url,
        json=message
      )
      
    print(response.text)


In [27]:
slack_webhook = "INSERT SLACKHOOK"

project = "evident-data-dev"

dataset_conventions = ('raw_', 'curated_', 'product_', 'temporary')
table_conventions = ('insurance_', 'banking_', 'payments_', 'omni_')

dataset_whitelist = {
                     "curated_taxonomies",
                     "curated_business_metrics",
                     "curated_hubspot", 
                     "raw_csv_imports",
                     "raw_hubspot",
                     "raw_business_metrics"        
                    }     
dataset_whitelist_sw = ("product_index")

table_whitelist = {}
table_whitelist_sw = ()

In [ ]:
fail_datasets = []
fail_tables = []

payload = {"project":project}

bq_client = bigquery.Client(project=project)
datasets = bq_client.list_datasets()

for dataset in datasets:
    dataset_id = dataset.dataset_id

    if dataset_id in dataset_whitelist or dataset_id.startswith(dataset_whitelist_sw):
        continue

    if not dataset_id.startswith(dataset_conventions):
        fail_datasets.append(f"\n{dataset_id}")
    
    tables = bq_client.list_tables(dataset_id)
    for table in tables:
        table_id = table.table_id

        if table_id in table_whitelist or table_id.startswith(table_whitelist_sw):
            continue

        if not table_id.startswith(table_conventions):
            fail_tables.append(f"\n{dataset_id}.{table_id}")

payload["failed_datasets"] = sorted(fail_datasets) if fail_datasets else "none"
payload["failed_tables"] = sorted(fail_tables) if fail_tables else "All passing"

send_slack_message(slack_webhook, payload)

{'project': 'evident-data-dev', 'failed_datasets': 'none', 'failed_tables': ['\ncurated_google_analytics.member_hub_events', '\ncurated_google_patents.patents', '\ncurated_google_scholar.ai_keyword_list', '\ncurated_google_scholar.authors', '\ncurated_google_scholar.authors_blacklist', '\ncurated_google_scholar.authors_whitelist', '\ncurated_google_scholar.company_affiliations_lookup', '\ncurated_google_scholar.papers', '\ncurated_google_scholar.research', '\ncurated_google_scholar.university_affiliations_lookup', '\ncurated_indicator_evidence.indicator_evidence', '\ncurated_linkedin_data.linkedin_posts_bank', '\ncurated_use_cases.filtered_use_cases', '\ncurated_use_cases.new_use_cases', '\nproduct_tracker_banking_use_cases.use_case_tracker', '\nraw_audit.log', '\nraw_google_patents.details', '\nraw_google_patents.failed_details', '\nraw_google_patents.failed_urls', '\nraw_google_patents.urls', '\nraw_google_patents.urls_all', '\nraw_google_scholar.404s', '\nraw_google_scholar.authors'